In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
sns.set()

## Raw Claims Data: EDA

In [4]:
death_claims = pd.read_csv("Death Claims Data.csv")#read raw death claims dataset

In [5]:
death_claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1066 entries, 0 to 1065
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   DOB                1066 non-null   object
 1   CLAIM REASON       1010 non-null   object
 2   CLAIM DATE         1066 non-null   object
 3   NOTIFICATION DATE  1066 non-null   object
dtypes: object(4)
memory usage: 33.4+ KB


In [6]:
from pandas_profiling import ProfileReport #run pandas profiler on raw death claims dataset
prof = ProfileReport(death_claims)
prof.to_file(output_file='Profile Output - Raw Death Claims Data.html')
             


C:\Users\jloftus\AppData\Local\Temp\ipykernel_16172\2437098761.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport #run pandas profiler on raw death claims dataset


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Feature Engineering & Feature Selection

### Drop N/As and Duplicates from Raw Death Claims data

In [7]:
death_claims = death_claims.dropna() 
death_claims = death_claims.drop_duplicates() 
death_claims.shape

(992, 4)

### Calculate a) delay between claim date and notification date and b) age of policyholder at claim date

In [8]:
death_claims['CLAIM DATE']= pd.DatetimeIndex(death_claims['CLAIM DATE'].values) 
death_claims['DOB']= pd.DatetimeIndex(death_claims['DOB'].values)
death_claims['NOTIFICATION DATE']= pd.DatetimeIndex(death_claims['NOTIFICATION DATE'].values)
death_claims['NOTIFICATION YEAR'] = pd.PeriodIndex(death_claims['NOTIFICATION DATE'], freq='Y')
death_claims['NOTIFICATION QUARTER'] = pd.PeriodIndex(death_claims['NOTIFICATION DATE'], freq='Q')
death_claims['DELAY'] = death_claims['NOTIFICATION DATE'] - death_claims['CLAIM DATE'] 
death_claims['AGE'] = (death_claims['CLAIM DATE']- death_claims['DOB'])


In [9]:
death_claims.shape

(992, 8)

In [10]:
death_claims.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 992 entries, 1 to 1065
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   DOB                   992 non-null    datetime64[ns] 
 1   CLAIM REASON          992 non-null    object         
 2   CLAIM DATE            992 non-null    datetime64[ns] 
 3   NOTIFICATION DATE     992 non-null    datetime64[ns] 
 4   NOTIFICATION YEAR     992 non-null    period[A-DEC]  
 5   NOTIFICATION QUARTER  992 non-null    period[Q-DEC]  
 6   DELAY                 992 non-null    timedelta64[ns]
 7   AGE                   992 non-null    timedelta64[ns]
dtypes: datetime64[ns](3), object(1), period[A-DEC](1), period[Q-DEC](1), timedelta64[ns](2)
memory usage: 69.8+ KB


In [11]:
avg_death_delay = np.mean(death_claims['DELAY'])
avg_death_delay

Timedelta('35 days 14:29:30.967741935')

In [12]:
death_claims['AGE'].sort_values(ascending = True)

860    9379 days
775    9548 days
737    9585 days
738    9585 days
756    9823 days
         ...    
192   31862 days
655   31956 days
479   32778 days
954   33251 days
261   35213 days
Name: AGE, Length: 992, dtype: timedelta64[ns]

In [13]:
death_claims['AGE'].sort_values(ascending = False)

261   35213 days
954   33251 days
479   32778 days
655   31956 days
192   31862 days
         ...    
756    9823 days
737    9585 days
738    9585 days
775    9548 days
860    9379 days
Name: AGE, Length: 992, dtype: timedelta64[ns]

In [14]:
from pandas_profiling import ProfileReport #Run Profile Report again
prof = ProfileReport(death_claims)
prof.to_file(output_file='Profile Output - Enhanced Claims Data.html')
            

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Calculate Number of Death Claims by Year and Number of Death Claims by Quarter

In [15]:
death_claims_by_year = pd.pivot_table(death_claims,index=['NOTIFICATION YEAR'],aggfunc={'CLAIM DATE': len}).rename(columns={'CLAIM DATE': 'Number of Death Claims'}) 
death_claims_by_year.plot()

<AxesSubplot:xlabel='NOTIFICATION YEAR'>

In [16]:
death_claims_by_q = pd.pivot_table(death_claims,index=['NOTIFICATION QUARTER'],aggfunc={'CLAIM DATE': len}).rename(columns={'CLAIM DATE': 'NUMBER OF QUARTERLY DEATH CLAIMS'})


In [17]:
death_claims_by_q
death_claims_by_q.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 78 entries, 2000Q4 to 2022Q2
Freq: Q-DEC
Data columns (total 1 columns):
 #   Column                            Non-Null Count  Dtype
---  ------                            --------------  -----
 0   NUMBER OF QUARTERLY DEATH CLAIMS  78 non-null     int64
dtypes: int64(1)
memory usage: 1.2 KB


In [18]:
death_claims_by_q.to_csv("Death Claims by Quarter.csv", index=True)

### Calculate Quarterly Death Claim Seasonal Variation

In [19]:
death_claims_by_year = pd.pivot_table(death_claims,index=['NOTIFICATION YEAR'],aggfunc={'CLAIM DATE': len}).rename(columns={'CLAIM DATE': 'NUMBER OF ANNUAL DEATH CLAIMS'})
death_claims_qy = death_claims_by_q.reset_index(level = ['NOTIFICATION QUARTER'])
death_claims_qy['NOTIFICATION YEAR'] = (death_claims_qy['NOTIFICATION QUARTER']).astype(str).str[:4]
death_claims_by_year = death_claims_by_year.reset_index(level = ['NOTIFICATION YEAR'])
death_claims_by_year['NOTIFICATION YEAR'] = death_claims_by_year['NOTIFICATION YEAR'].astype(str)
death_claims_qy = pd.merge(
    left=death_claims_qy,
    right=death_claims_by_year,   
    how='left'
)

death_claims_qy['CLAIMS SEASONAL VARIATION'] = death_claims_qy['NUMBER OF QUARTERLY DEATH CLAIMS']/death_claims_qy['NUMBER OF ANNUAL DEATH CLAIMS']
death_claims_qy['CLAIMS SEASONAL VARIATION'] = death_claims_qy['CLAIMS SEASONAL VARIATION']-0.25
death_claims_qy = death_claims_qy[(death_claims_qy['NOTIFICATION YEAR'].astype(int)>2004) & (death_claims_qy['NOTIFICATION YEAR'].astype(int)<2022) ]
death_claims_qy.tail(12)

,NOTIFICATION QUARTER,NUMBER OF QUARTERLY DEATH CLAIMS,NOTIFICATION YEAR,NUMBER OF ANNUAL DEATH CLAIMS,CLAIMS SEASONAL VARIATION
64,2019Q1,18,2019,65,0.026923
65,2019Q2,12,2019,65,-0.065385
66,2019Q3,21,2019,65,0.073077
67,2019Q4,14,2019,65,-0.034615
68,2020Q1,22,2020,74,0.047297
69,2020Q2,20,2020,74,0.020270
70,2020Q3,17,2020,74,-0.020270
71,2020Q4,15,2020,74,-0.047297
72,2021Q1,22,2021,72,0.055556
73,2021Q2,20,2021,72,0.027778


In [20]:
death_claims_qy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 8 to 75
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype        
---  ------                            --------------  -----        
 0   NOTIFICATION QUARTER              68 non-null     period[Q-DEC]
 1   NUMBER OF QUARTERLY DEATH CLAIMS  68 non-null     int64        
 2   NOTIFICATION YEAR                 68 non-null     object       
 3   NUMBER OF ANNUAL DEATH CLAIMS     68 non-null     int64        
 4   CLAIMS SEASONAL VARIATION         68 non-null     float64      
dtypes: float64(1), int64(2), object(1), period[Q-DEC](1)
memory usage: 3.2+ KB


In [21]:
death_claims_qy['Q'] = (death_claims_qy['NOTIFICATION QUARTER'].astype(str)).str[-2:]
death_claims_q_avg = (death_claims_qy.groupby('Q')['CLAIMS SEASONAL VARIATION'].mean().to_frame()).rename(columns={'CLAIMS SEASONAL VARIATION': 'Average Seasonal Variation - Death Claims'})
death_claims_q_avg = death_claims_q_avg.reset_index()
death_claims_q_avg

,Q,Average Seasonal Variation - Death Claims
0,Q1,0.024034
1,Q2,-0.016384
2,Q3,-0.014277
3,Q4,0.006627


In [22]:
sd_death_claims = np.std(death_claims_q_avg['Average Seasonal Variation - Death Claims']) #calculate standard deviation of death claims seasonal variation
sd_death_claims

0.016536546198098367

In [23]:
plt.rcParams["figure.figsize"] = (16,8)
sns.barplot(x = 'Q', y = 'Average Seasonal Variation - Death Claims', data = death_claims_q_avg)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Calendar Year Quarter", size = 16)
plt.ylabel("Average Seasonal Variation",size = 16)
#plt.title('Average Seasonal Variation by Calendar Year Quarter (2005 to 2021) : Death Claims', size = 16, fontweight = "bold")
plt.legend(fontsize = 14)
plt.show()

C:\Users\jloftus\AppData\Local\Temp\ipykernel_16172\2729392185.py:9: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Seasonal Decomposition - Number of Death Claims

In [24]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

series = death_claims_qy.set_index('NOTIFICATION QUARTER')
series = series['NUMBER OF QUARTERLY DEATH CLAIMS']
series.interpolate(inplace = True)
series.index=series.index.to_timestamp()
result = seasonal_decompose(series, model='additive',period =12)
print("Seasonal Decomposition: Number of Death Claims (2005 to 2021)")
result.plot()
plt.show()

Seasonal Decomposition: Number of Death Claims (2005 to 2021)


C:\Users\jloftus\AppData\Local\Temp\ipykernel_16172\2709549648.py:11: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Augmented Dickey Fuller Test - Number of Death Claims

In [25]:
X = death_claims_qy.set_index('NOTIFICATION QUARTER')
X = death_claims_qy['NUMBER OF QUARTERLY DEATH CLAIMS']
result = adfuller(X)
print('ADF Statistic (Number of Death Claims notified per quarter): %f' % result[0])
print('p-value (Number of Death Claims notified per quarter): %f' % result[1])
print('Critical Values (Number of Death Claims notified per quarter):')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic (Number of Death Claims notified per quarter): -6.487336
p-value (Number of Death Claims notified per quarter): 0.000000
Critical Values (Number of Death Claims notified per quarter):
	1%: -3.532
	5%: -2.906
	10%: -2.590


### Augmented Dickey Fuller Test - Death Claim Seasonal Variation

In [26]:
X = death_claims_qy.set_index('NOTIFICATION QUARTER')
X = death_claims_qy['CLAIMS SEASONAL VARIATION']
result = adfuller(X)
print('ADF Statistic (Death Claim Quarterly Seasonal Variation): %f' % result[0])
print('p-value (Death Claim Quarterly Seasonal Variation): %f' % result[1])
print('Critical Values (Death Claim Quarterly Seasonal Variation):')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic (Death Claim Quarterly Seasonal Variation): -4.810408
p-value (Death Claim Quarterly Seasonal Variation): 0.000052
Critical Values (Death Claim Quarterly Seasonal Variation):
	1%: -3.544
	5%: -2.911
	10%: -2.593


### Autocorrelation Plot - Death Claim Seasonal Variation

In [27]:
from pandas.plotting import autocorrelation_plot
plt.rcParams.update({'figure.figsize':(9,5), 'figure.dpi':120})
autocorrelation_plot(death_claims_qy['CLAIMS SEASONAL VARIATION'].tolist())

<AxesSubplot:xlabel='Lag', ylabel='Autocorrelation'>

In [28]:
series = death_claims_qy.set_index('NOTIFICATION QUARTER')
series = series['CLAIMS SEASONAL VARIATION']
series.interpolate(inplace = True)
series.index=series.index.to_timestamp()
result = seasonal_decompose(series, model='additive',period =12)
print("Seasonal Decomposition: Quarterly Death Claim Seasonal Variation (2005 to 2021)")
result.plot()
plt.show()

Seasonal Decomposition: Quarterly Death Claim Seasonal Variation (2005 to 2021)


C:\Users\jloftus\AppData\Local\Temp\ipykernel_16172\2532690095.py:8: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Export Cleaned Death Claims Dataset for Modelling

### Death Claims Dataset including Covid-19 Claims

In [29]:
#write csv file
death_claims_output = death_claims_qy.drop(columns =['NUMBER OF QUARTERLY DEATH CLAIMS', 'NOTIFICATION YEAR','NUMBER OF ANNUAL DEATH CLAIMS'] )
death_claims_output.to_csv("Death Claim Seasonality.csv", index=False)

In [30]:
death_claims_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 8 to 75
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype        
---  ------                     --------------  -----        
 0   NOTIFICATION QUARTER       68 non-null     period[Q-DEC]
 1   CLAIMS SEASONAL VARIATION  68 non-null     float64      
 2   Q                          68 non-null     object       
dtypes: float64(1), object(1), period[Q-DEC](1)
memory usage: 2.1+ KB


In [31]:
death_claims_output.plot()

<AxesSubplot:>

### Death Claims Dataset excluding Covid-19 Claims

In [32]:
death_reason = list(death_claims['CLAIM REASON'].unique())
death_reason

['Suicide',
 'Heart Attack',
 'pancreatic cancer',
 'Other Death Reason',
 'Colon',
 'Lung',
 'aortic dissection',
 'Breast Cancer',
 'Ischaemic cardiomyopathy',
 'Major Organ',
 'Stroke',
 "Alzheimer's Disease (Pre-senile Dementia)",
 'Bilateral Bronchopneumoni',
 'Motor Neurone Disease',
 'Road Traffic Accident',
 'Broncho-pneumonia',
 'Brain and other cancers',
 'Brain Haemorrhage',
 'Covid 19',
 'Ovarian /Uterine',
 'Septicaemia',
 'cardiomyopathy',
 'Kidney Failure',
 'Myocardial Degeneration',
 'Cardiac Arrhythmia',
 'Pancreatic Cancer',
 'Leukaemias',
 'Pneumonia/Dementia',
 'Hepatic Encephalopathy',
 "Parkinson's Disease",
 'Lymphomas',
 'Skin',
 'septicaemia',
 'End Stage Dementia',
 'Oesphagal Cancer',
 'Bilateral Pneumonia',
 'Cirrhosis',
 'IHD',
 'Respiratory Failure',
 'Cholangiocarcinoma',
 'Haemorrhage',
 'Injuries fallowing a fall',
 'Other Illnesses',
 'Oesophageal Cancer',
 'Prostate',
 'ischaemic Heart Disease',
 'Blood Poisoning',
 'Brain Aneurysm',
 'Fell and Fract

In [33]:
covid_death_claims = death_claims[(death_claims['CLAIM REASON'] == 'Covid 19') & (death_claims['CLAIM REASON'] == 'Covid Pneumonia') & (death_claims['CLAIM REASON'] == 'covid 19') & (death_claims['CLAIM REASON'] == 'Mrsa/Covid')  ]
covid_death_claims.shape

(0, 8)

In [34]:
death_claims_ex_covid = death_claims[(death_claims['CLAIM REASON'] != 'Covid 19') & (death_claims['CLAIM REASON'] != 'Covid Pneumonia') & (death_claims['CLAIM REASON'] != 'covid 19') & (death_claims['CLAIM REASON'] != 'Mrsa/Covid')  ]
death_claims_ex_covid.shape

(987, 8)

In [35]:
death_claims_by_q = pd.pivot_table(death_claims_ex_covid,index=['NOTIFICATION QUARTER'],aggfunc={'CLAIM DATE': len}).rename(columns={'CLAIM DATE': 'NUMBER OF QUARTERLY DEATH CLAIMS'})
death_claims_by_year = pd.pivot_table(death_claims_ex_covid,index=['NOTIFICATION YEAR'],aggfunc={'CLAIM DATE': len}).rename(columns={'CLAIM DATE': 'NUMBER OF ANNUAL DEATH CLAIMS'})
death_claims_qy = death_claims_by_q.reset_index(level = ['NOTIFICATION QUARTER'])
death_claims_qy['NOTIFICATION YEAR'] = (death_claims_qy['NOTIFICATION QUARTER']).astype(str).str[:4]
death_claims_by_year = death_claims_by_year.reset_index(level = ['NOTIFICATION YEAR'])
death_claims_by_year['NOTIFICATION YEAR'] = death_claims_by_year['NOTIFICATION YEAR'].astype(str)
death_claims_qy = pd.merge(
    left=death_claims_qy,
    right=death_claims_by_year,   
    how='left'
)

death_claims_qy['CLAIMS SEASONAL VARIATION'] = death_claims_qy['NUMBER OF QUARTERLY DEATH CLAIMS']/death_claims_qy['NUMBER OF ANNUAL DEATH CLAIMS']
death_claims_qy['CLAIMS SEASONAL VARIATION'] = death_claims_qy['CLAIMS SEASONAL VARIATION']-0.25
death_claims_qy = death_claims_qy[(death_claims_qy['NOTIFICATION YEAR'].astype(int)>2004) & (death_claims_qy['NOTIFICATION YEAR'].astype(int)<2022) ]
death_claims_qy.tail(12)

,NOTIFICATION QUARTER,NUMBER OF QUARTERLY DEATH CLAIMS,NOTIFICATION YEAR,NUMBER OF ANNUAL DEATH CLAIMS,CLAIMS SEASONAL VARIATION
64,2019Q1,18,2019,65,0.026923
65,2019Q2,12,2019,65,-0.065385
66,2019Q3,21,2019,65,0.073077
67,2019Q4,14,2019,65,-0.034615
68,2020Q1,22,2020,72,0.055556
69,2020Q2,18,2020,72,0.000000
70,2020Q3,17,2020,72,-0.013889
71,2020Q4,15,2020,72,-0.041667
72,2021Q1,21,2021,69,0.054348
73,2021Q2,19,2021,69,0.025362


In [36]:
#write csv file
death_claims_output_ex_covid = death_claims_qy.drop(columns =['NUMBER OF QUARTERLY DEATH CLAIMS', 'NOTIFICATION YEAR','NUMBER OF ANNUAL DEATH CLAIMS'] )
death_claims_output_ex_covid.to_csv("Death Claim Seasonality - ex Covid Claims.csv", index=False)

In [37]:
death_claims_output_ex_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 8 to 75
Data columns (total 2 columns):
 #   Column                     Non-Null Count  Dtype        
---  ------                     --------------  -----        
 0   NOTIFICATION QUARTER       68 non-null     period[Q-DEC]
 1   CLAIMS SEASONAL VARIATION  68 non-null     float64      
dtypes: float64(1), period[Q-DEC](1)
memory usage: 1.6 KB


## Raw CSO Data - EDA

In [154]:
df = pd.read_csv('VSAQ2.20220608T130600.csv') # read raw cso deaths data
df.tail(15)


,STATISTIC,Quarter,Age at Death,UNIT,VALUE
3705,Deaths Registered Provisional,2021Q4,Under 1 Year,Number,62.0
3706,Deaths Registered Provisional,2021Q4,4 Weeks to 1 Year,Number,16.0
3707,Deaths Registered Provisional,2021Q4,1 Week to 4 Weeks,Number,11.0
3708,Deaths Registered Provisional,2021Q4,Under 7 Days,Number,35.0
3709,Deaths Registered Provisional,2021Q4,1 to 7 Days,Number,18.0
3710,Deaths Registered Provisional,2021Q4,Under 1 Day,Number,17.0
3711,Deaths Registered Provisional,2021Q4,1 to 4 Years,Number,1.0
3712,Deaths Registered Provisional,2021Q4,5 to 14 Years,Number,10.0
3713,Deaths Registered Provisional,2021Q4,15 to 24 Years,Number,52.0
3714,Deaths Registered Provisional,2021Q4,25 to 34 Years,Number,78.0


In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3720 entries, 0 to 3719
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STATISTIC     3720 non-null   object 
 1   Quarter       3720 non-null   object 
 2   Age at Death  3720 non-null   object 
 3   UNIT          3720 non-null   object 
 4   VALUE         3248 non-null   float64
dtypes: float64(1), object(4)
memory usage: 145.4+ KB


In [156]:
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file='Profile Output - Raw CSO Deaths Data.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Drop N/As

In [157]:
df = df.dropna()

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3248 entries, 0 to 3719
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STATISTIC     3248 non-null   object 
 1   Quarter       3248 non-null   object 
 2   Age at Death  3248 non-null   object 
 3   UNIT          3248 non-null   object 
 4   VALUE         3248 non-null   float64
dtypes: float64(1), object(4)
memory usage: 152.2+ KB


In [159]:
df['Age at Death'].unique()

array(['Under 1 Year', '1 to 4 Years', '5 to 14 Years', '15 to 24 Years',
       '25 to 34 Years', '35 to 44 Years', '45 to 54 Years',
       '55 to 64 Years', '65 to 74 Years', '75 Years and Over',
       '4 Weeks to 1 Year', '1 Week to 4 Weeks', 'Under 7 Days',
       '1 to 7 Days', 'Under 1 Day'], dtype=object)

## Feature Engineering & Feature Selection

In [160]:
df['Quarter']= pd.DatetimeIndex(df['Quarter'].values)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3248 entries, 0 to 3719
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   STATISTIC     3248 non-null   object        
 1   Quarter       3248 non-null   datetime64[ns]
 2   Age at Death  3248 non-null   object        
 3   UNIT          3248 non-null   object        
 4   VALUE         3248 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 152.2+ KB


C:\Users\jloftus\AppData\Local\Temp\ipykernel_16172\3213116988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Quarter']= pd.DatetimeIndex(df['Quarter'].values)


In [161]:
cso_df = df[(df['Quarter'].dt.year>=2005)] #remove deaths prior to 2005

In [162]:
cso_df_75 = cso_df[(cso_df['Age at Death'] == "75 Years and Over")]
cso_df_75

,STATISTIC,Quarter,Age at Death,UNIT,VALUE
2714,Deaths Registered Provisional,2005-01-01,75 Years and Over,Number,4616.0
2729,Deaths Registered Provisional,2005-04-01,75 Years and Over,Number,4440.0
2744,Deaths Registered Provisional,2005-07-01,75 Years and Over,Number,3699.0
2759,Deaths Registered Provisional,2005-10-01,75 Years and Over,Number,3663.0
2774,Deaths Registered Provisional,2006-01-01,75 Years and Over,Number,4702.0
...,...,...,...,...,...
3659,Deaths Registered Provisional,2020-10-01,75 Years and Over,Number,4642.0
3674,Deaths Registered Provisional,2021-01-01,75 Years and Over,Number,6550.0
3689,Deaths Registered Provisional,2021-04-01,75 Years and Over,Number,4339.0
3704,Deaths Registered Provisional,2021-07-01,75 Years and Over,Number,5112.0


In [163]:
cso_df_75 = cso_df[(cso_df['Age at Death'] == "75 Years and Over")]
cso_df_75['VALUE'].sum()/cso_df['VALUE'].sum() #calculate percentage of total deaths that were in the 75 and over category

0.6149444697916852

In [164]:
df_25 = df[(df['Age at Death'] == "Under 1 Year") | (df['Age at Death'] == "4 Weeks to 1 Year") | (df['Age at Death'] == "1 Week to 4 Weeks") | (df['Age at Death'] == "Under 7 Days") | (df['Age at Death'] == "1 to 7 Days") | (df['Age at Death'] == "Under 1 Day") | (df['Age at Death'] == "1 to 4 Years") | (df['Age at Death'] == "5 to 14 Years") | (df['Age at Death'] == "15 to 24 Years")]
df_25['VALUE'].sum()/df['VALUE'].sum() #calculate percentage of total deaths that were in the under 25 category

0.04911130260193374

In [165]:
#Remove Death Age Groups that I'm not using
cso_df = cso_df[(cso_df['Age at Death'] != "Under 1 Year") & (cso_df['Age at Death'] != "4 Weeks to 1 Year") & (cso_df['Age at Death'] != "1 Week to 4 Weeks") & (cso_df['Age at Death'] != "Under 7 Days") & (cso_df['Age at Death'] != "1 to 7 Days") & (cso_df['Age at Death'] != "Under 1 Day") & (cso_df['Age at Death'] != "1 to 4 Years") & (cso_df['Age at Death'] != "5 to 14 Years") & (cso_df['Age at Death'] != "15 to 24 Years") & (cso_df['Age at Death'] != "75 Years and Over")  ]


In [166]:
cso_deaths_by_q = cso_df.groupby('Quarter')['VALUE'].sum()
cso_deaths_by_q = cso_deaths_by_q.to_frame()
cso_deaths_by_q.sum() #calculate total number of CSO deaths


VALUE    179774.0
dtype: float64

In [167]:
df_75['Quarter']= pd.DatetimeIndex(df_75['Quarter'].values)
deaths_by_q_75 = df_75[df_75['Quarter'].dt.year>=2005].groupby('Quarter')['VALUE'].sum()
deaths_by_q_75= deaths_by_q_75.to_frame()
df_under_25['Quarter']= pd.DatetimeIndex(df_under_25['Quarter'].values)
deaths_by_q_under_25 = df_under_25[df_under_25['Quarter'].dt.year>=2005].groupby('Quarter')['VALUE'].sum()
deaths_by_q_under_25 = deaths_by_q_under_25.to_frame()

In [168]:
sns.set()
from matplotlib.pyplot import figure
figure(figsize=(16, 8))
plt.plot(deaths_by_q_75.VALUE, color = "red", label = '75 Years olds and over')
plt.plot(cso_deaths_by_q.VALUE, color = "steelblue", label = '25 to 74 Year olds')
plt.plot(deaths_by_q_under_25.VALUE, color='pink', label = 'Under 25 years old')
plt.xlabel('Number of Irish Population Deaths Registered per Quarter', fontsize = 20)
plt.ylabel('Year', fontsize = 20)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.legend(fontsize = 16)
plt.show()

C:\Users\jloftus\AppData\Local\Temp\ipykernel_16172\3908042948.py:12: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Calculate CSO Deaths Seasonal Variation

In [169]:
#Sum by Quarter
cso_deaths_by_q.rename(columns = {'VALUE':'Quarterly Deaths'}, inplace = True)
cso_deaths_by_qy = cso_deaths_by_q
cso_deaths_by_qy['Year'] =  pd.PeriodIndex(cso_deaths_by_qy.index, freq='Y')
cso_deaths_by_qy = cso_deaths_by_qy.reset_index(level = ['Quarter'])
#Sum by Year
cso_deaths_by_y = pd.pivot_table(cso_deaths_by_q,index=['Year'],aggfunc={'Quarterly Deaths': sum}).rename(columns={'Quarterly Deaths': 'Annual Deaths'})
cso_deaths_by_y = cso_deaths_by_y.reset_index(level = ['Year'])

cso_deaths_qy = pd.merge(
    left=cso_deaths_by_qy,
    right=cso_deaths_by_y,   
    how='left'
)

cso_deaths_qy['CSO SEASONAL VARIATION'] = cso_deaths_qy['Quarterly Deaths']/cso_deaths_qy['Annual Deaths']
cso_deaths_qy['CSO SEASONAL VARIATION'] = cso_deaths_qy['CSO SEASONAL VARIATION']-0.25
cso_deaths_qy['Quarter'] = pd.PeriodIndex(cso_deaths_qy['Quarter'], freq='Q')
cso_deaths_qy['Q'] = (cso_deaths_qy['Quarter'].astype(str)).str[-2:]
cso_deaths_q_avg = (cso_deaths_qy.groupby('Q')['CSO SEASONAL VARIATION'].mean().to_frame()).rename(columns={'CSO SEASONAL VARIATION': 'Average Seasonal Variation - CSO Deaths'})
cso_deaths_q_avg = cso_deaths_q_avg.reset_index()
cso_deaths_q_avg

,Q,Average Seasonal Variation - CSO Deaths
0,Q1,0.012098
1,Q2,-0.000664
2,Q3,-0.003569
3,Q4,-0.007865


In [170]:
sd_cso_deaths = np.std(cso_deaths_q_avg['Average Seasonal Variation - CSO Deaths']) #calculate standard deviation of CSO deaths seasonal variation
sd_cso_deaths

0.007439768314107906

In [171]:
sns.set()
plt.rcParams["figure.figsize"] = (16,8)
sns.barplot(x = 'Q', y = 'Average Seasonal Variation - CSO Deaths', data = cso_deaths_q_avg)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Calendar Year Quarter", size = 16)
plt.ylabel("Average Seasonal Variation",size = 16)
#plt.title('Average Seasonal Variation by Calendar Year Quarter (2005 to 2021) : Death Claims', size = 16, fontweight = "bold")
plt.legend(fontsize = 14)
plt.show()

C:\Users\jloftus\AppData\Local\Temp\ipykernel_16172\302600278.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Seasonal Decomposition - Number of Quarterly Deaths Age 25 to 74

In [172]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

series = cso_deaths_qy.set_index('Quarter')
series = series['Quarterly Deaths']
series.interpolate(inplace = True)
series.index=series.index.to_timestamp()
result = seasonal_decompose(series, model='additive',period =12)
print("Seasonal Decomposition: CSO Irish Population Deaths Age 25 to 74 (2005 to 2021)")
result.plot()
plt.show()

Seasonal Decomposition: CSO Irish Population Deaths Age 25 to 74 (2005 to 2021)


C:\Users\jloftus\AppData\Local\Temp\ipykernel_16172\533751796.py:11: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Augmented Dickey Fuller Test -  Seasonal Variation in Number of Deaths Age 25 to 74

In [173]:
X =cso_deaths_qy.set_index('Quarter')
X = cso_deaths_qy['CSO SEASONAL VARIATION']
result = adfuller(X)
print('ADF Statistic (CSO Deaths Seasonal Variation (2005 to 2021) - Age 25 to 74): %f' % result[0])
print('p-value (CSO Deaths Seasonal Variation (2005 to 2021) - Age 25 to 74): %f' % result[1])
print('Critical Values (CSO Deaths Seasonal Variation (2005 to 2021) - Age 25 to 74):')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))


ADF Statistic (CSO Deaths Seasonal Variation (2005 to 2021) - Age 25 to 74): -2.738597
p-value (CSO Deaths Seasonal Variation (2005 to 2021) - Age 25 to 74): 0.067594
Critical Values (CSO Deaths Seasonal Variation (2005 to 2021) - Age 25 to 74):
	1%: -3.544
	5%: -2.911
	10%: -2.593


## Export Cleaned CSO Deaths Dataset

In [174]:
cso_deaths_output = cso_deaths_qy.drop(columns =['Quarterly Deaths', 'Year','Annual Deaths','Q'] )

In [175]:
cso_deaths_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 0 to 67
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype        
---  ------                  --------------  -----        
 0   Quarter                 68 non-null     period[Q-DEC]
 1   CSO SEASONAL VARIATION  68 non-null     float64      
dtypes: float64(1), period[Q-DEC](1)
memory usage: 1.6 KB


In [176]:
#write csv file
cso_deaths_output.to_csv("CSO Deaths Seasonality.csv", index=False)

## Compare Claim Deaths Seasonal Variation with CSO Deaths Seasonal Variation

In [177]:
cso_deaths_q_avg

,Q,Average Seasonal Variation - CSO Deaths
0,Q1,0.012098
1,Q2,-0.000664
2,Q3,-0.003569
3,Q4,-0.007865


In [178]:
death_claims_q_avg

,Q,Average Seasonal Variation - Death Claims
0,Q1,0.024034
1,Q2,-0.016384
2,Q3,-0.014277
3,Q4,0.006627


In [179]:
combined_deaths_q_avg = pd.merge(
    left=death_claims_q_avg,
    right=cso_deaths_q_avg,   
    how='left'
)
#combined_deaths_q_avg = combined_deaths_q_avg.set_index('Q')

In [180]:
combined_deaths_q_avg

,Q,Average Seasonal Variation - Death Claims,Average Seasonal Variation - CSO Deaths
0,Q1,0.024034,0.012098
1,Q2,-0.016384,-0.000664
2,Q3,-0.014277,-0.003569
3,Q4,0.006627,-0.007865


In [181]:
combined_deaths_q_avg = combined_deaths_q_avg.melt(id_vars = "Q")
combined_deaths_q_avg

,Q,variable,value
0,Q1,Average Seasonal Variation - Death Claims,0.024034
1,Q2,Average Seasonal Variation - Death Claims,-0.016384
2,Q3,Average Seasonal Variation - Death Claims,-0.014277
3,Q4,Average Seasonal Variation - Death Claims,0.006627
4,Q1,Average Seasonal Variation - CSO Deaths,0.012098
5,Q2,Average Seasonal Variation - CSO Deaths,-0.000664
6,Q3,Average Seasonal Variation - CSO Deaths,-0.003569
7,Q4,Average Seasonal Variation - CSO Deaths,-0.007865


In [182]:
sns.set()
plt.rcParams["figure.figsize"] = (16,8)
sns.barplot(x = 'Q', y = combined_deaths_q_avg.value, hue = combined_deaths_q_avg.variable, data = combined_deaths_q_avg)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel("Calendar Year Quarter", size = 20)
plt.ylabel("Average Quarterly Seasonal Variation",size = 20)
#plt.title('Average Seasonal Variation by Calendar Year Quarter (2005 to 2021) : Death Claims', size = 16, fontweight = "bold")
plt.legend(fontsize = 14)
plt.show()

C:\Users\jloftus\AppData\Local\Temp\ipykernel_16172\1807715562.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
